In [9]:
from anytree import RenderTree
from cannonball.nodes import Task, Bullet

# Create root node
root = Task("ProjectX")

# Create child Tasks
task1 = Task("Implement API", parent=root)
task2 = Task("Write Documentation", parent=root)

# Create nested children
subtask1 = Task("Design Endpoints", parent=task1)
subtask2 = Task("Implement Authentication", parent=task1)
subtask3 = Task("Backend Requirements", parent=subtask1)
bullet = Bullet("API", parent=subtask1)
subtask4 = Task("API Implementation", parent=bullet)
subtask5 = Task("API Testing", parent=bullet)

subtask3.complete()
subtask4.complete()
# subtask5.complete()
# print(subtask1.complete())

# Print the tree
for pre, _, node in RenderTree(root):
    print(f"{pre}{node}")

[/] ProjectX
├── [/] Implement API
│   ├── [✓] Design Endpoints
│   │   ├── [✓] Backend Requirements
│   │   └── API
│   │       ├── [✓] API Implementation
│   │       └── [ ] API Testing
│   └── [ ] Implement Authentication
└── [ ] Write Documentation


In [10]:
from cannonball.nodes import Question, Answer, Bullet, Task

# Create root node
question = Question("What graph library should I use?")

bullet1 = Bullet("NetworkX", parent=question)
task1 = Task("Check documentation", parent=bullet1)
bullet2 = Bullet("Graph-tool", parent=question)
bullet3 = Bullet("igraph", parent=question)

# task1.start()
# task1.complete()
answer = Answer("NetworkX", parent=question)
# decision = Decision("NetworkX", parent=question)
# decision.state = NodeState.COMPLETED

# Print the tree
for pre, _, node in RenderTree(question):
    print(f"{pre}{node}")

[?✓] What graph library should I use?
├── NetworkX
│   └── [ ] Check documentation
├── Graph-tool
├── igraph
└── [A] NetworkX


In [1]:
from cannonball.nodes import parse_markdown
from anytree import RenderTree
from anytree.search import find_by_attr

markdown = """\
        - [ ] Task 1
            - [ ] Task 2
                - I'm just a bullet
                - [ ] Task 2.1
            - [ ] Task 3
                - [?] How do I solve this?
                    - [A] I don't
                        - [ ] Task 4
"""

root = parse_markdown(markdown)

find_by_attr(root, "Task 2").complete()
find_by_attr(root, "Task 2.1").start()
find_by_attr(root, "Task 4").block()

# Print the tree
for pre, _, node in RenderTree(root):
    print(f"{pre}{node}")

[!] Task 1
├── [/] Task 2
│   ├── I'm just a bullet
│   └── [/] Task 2.1
└── [!] Task 3
    └── [?!] How do I solve this?
        └── [A] I don't
            └── [!] Task 4


In [1]:
from anytree import Node

root = Node("root")
child = Node("child", parent=root)
# This will raise LoopError:
root.parent = child

LoopError: Cannot set parent. Node('/root') is parent of Node('/root/child').